<a href="https://colab.research.google.com/github/bpratham2001/Servilia/blob/main/notebooks/fine_tune_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nn = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
#nn = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"
!pip install transformers
!pip install datasets
cats_full = ['abstract_algebra', 'anatomy', 'astronomy', 'business_ethics',
        'clinical_knowledge', 'college_biology', 'college_chemistry',
        'college_computer_science', 'college_mathematics', 'college_medicine',
        'college_physics', 'computer_security', 'conceptual_physics', 'econometrics',
        'electrical_engineering', 'elementary_mathematics', 'formal_logic',
        'global_facts', 'high_school_biology', 'high_school_chemistry',
        'high_school_computer_science', 'high_school_european_history',
        'high_school_geography', 'high_school_government_and_politics',
        'high_school_macroeconomics', 'high_school_mathematics',
        'high_school_microeconomics', 'high_school_physics',
        'high_school_psychology', 'high_school_statistics',
        'high_school_us_history', 'high_school_world_history',
        'human_aging', 'human_sexuality', 'international_law',
        'jurisprudence', 'logical_fallacies', 'machine_learning',
        'management', 'marketing', 'medical_genetics', 'miscellaneous',
        'moral_disputes', 'moral_scenarios', 'nutrition', 'philosophy',
        'prehistory', 'professional_accounting', 'professional_law',
        'professional_medicine', 'professional_psychology', 'public_relations',
        'security_studies', 'sociology', 'us_foreign_policy', 'virology', 'world_religions']

cats = ['abstract_algebra', 'elementary_mathematics', 'formal_logic', 'global_facts',
        'high_school_european_history', 'high_school_geography', 'high_school_government_and_politics',
        'high_school_psychology', 'human_aging', 'human_sexuality',
        'logical_fallacies', 'machine_learning', 'miscellaneous',
        'moral_disputes', 'moral_scenarios', 'philosophy', 'professional_psychology', 'sociology', 'world_religions']

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00


In [ ]:
"""
from datasets import load_dataset

d = {}
for i in range(len(cats)):
  d[cats[i]] = load_dataset('cais/mmlu', cats[i])
#d.filter(lambda x: x["subject"] in cats)
"""

In [2]:
import pandas as pd
file =('wine-tastings-guide.txt')
df1 = pd.read_csv(file, sep = ' - ')
file =('FR-IT.txt')
df2 = pd.read_csv(file, sep = ' - ')
file =('grapes.txt')
df3 = pd.read_csv(file, sep = ' - ')
file =('wine-enthusiast.txt') # Questions already framed
df4 = pd.read_csv(file, sep = ' - ')
file =('essence-of-wine.txt') # Questions already framed
df5 = pd.read_csv(file, sep = ' - ')

<ipython-input-2-510a31fafacc>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df1 = pd.read_csv(file, sep = ' - ')
<ipython-input-2-510a31fafacc>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df2 = pd.read_csv(file, sep = ' - ')
<ipython-input-2-510a31fafacc>:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df3 = pd.read_csv(file, sep = ' - ')
<ipython-input-2-510a31fafacc>:9: ParserWarning: Falling back to

In [3]:
print("total data points available: " + str(df1.shape[0]+df2.shape[0]+df3.shape[0]+df4.shape[0]+df5.shape[0]))

total data points available: 275


In [4]:
#target format
# how to train on df? yahma/alpaca-cleaned
sample = pd.read_json("hf://datasets/yahma/alpaca-cleaned/alpaca_data_cleaned.json")
from random import randint as r
# prompts
tastes = ["Tell me more about wines that are", "Describe to me wines that feel",
          "I would like to know more about wines considered", "I want to know more about wines that are",
          "On wines, tell me the meaning of", "Explain, in wine terminology,", "Elaborate on wines that feel", "Enlighten me on wines that feel",
          "Educate me on wines that are", "I don't understand wines that are"]
tastes_q = ["What does it mean when a wine is", "How can a wine be", "How are wines considered", "How are wines"]
tastes_q2 = ["Why does", "How can", "Why would", "How does", "What does it mean if"]
tastes_q3 = ["feel ", "taste ", "seem "]
other = ["Tell me more about", "Describe", "I would like to know more about", "I want to know more about",
         "Tell me about", "Explain", "Elaborate on", "Enlighten me on", "Educate me on", "Recommend me ", "Give me a recommendation regarding "]
what_singular = ["What is", "What's"]

#unused
what_plural = ["What are"]
how_singular = ["How does", "How is", "How can", "How come"]
how_plural = ["How do", "How are", "How can", "How come"]
where_singular = ["Where is", "What is the location of", "Where can I find", "How do I come across"]
where_plural = ["Where are", "What are the locations of"]
why_plural = ["Why do", "Why are"]

# function to construct question/answers from blogs/articles
def construct_prompt(row, rng, q_type=0):
  if q_type == 0:#tastes
    if rng == 0:
      instruction = str(tastes[r(0, len(tastes)-1)] + " " +row['word']+".")
    elif rng == 1:
      instruction = str(tastes_q[r(0, len(tastes_q)-1)] + " " +row['word']+"?")
    else:
      instruction = str(tastes_q2[r(0, len(tastes_q2)-2)] + " a wine "+ tastes_q3[r(0,len(tastes_q3)-1)] +row['word']+"?")
  elif q_type == 1:#regions
    if rng == 0:
      instruction = str(other[r(0, len(other)-1)] + " " +row['word']+".")
    elif rng == 1:
      instruction = str("How is " +row['word']+"?")
    else:
      instruction = str(what_singular[r(0, len(what_singular)-1)] + " " +row['word']+" like?")
  else:#grapes
    if rng == 0:
      instruction = str(other[r(0, len(other)-1)] + " " +row['word']+" grapes.")
    elif rng == 1:
      instruction = str(what_singular[r(0, len(what_singular)-1)] + " " +row['word']+"?")
    else:
      instruction = str("How are the wines that are made from " +row['word']+" grapes?")
  output = row['description']
  return [instruction, output]

sample.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


,instruction,input,output
0,Give three tips for staying healthy.,,1. Eat a balanced and nutritious diet: Make su...
1,What are the three primary colors?,,"The three primary colors are red, blue, and ye..."
2,Describe the structure of an atom.,,An atom is the basic building block of all mat...
3,How can we reduce air pollution?,,There are several ways to reduce air pollution...
4,Pretend you are a project manager of a constru...,,I had to make a difficult decision when I was ...


In [5]:
# formatting data into alpaca format
df0 = pd.DataFrame(columns=['instruction', 'input', 'output'])
#construct_prompt(row, rng, q_type=0)
lst = [df1, df2, df3]
for i in range(len(lst)):
  for index,row in lst[i].iterrows():
    a = construct_prompt(row, r(0,2), i)
    df0.loc[len(df0)] = [a[0],'',a[1]]
for j in [df4, df5]:
  for index,row in j.iterrows():
    df0.loc[len(df0)] = [row['question'],'',row['answer']]
  if df0.shape[0] < (df1.shape[0]+df2.shape[0]+df3.shape[0]+df4.shape[0]+df5.shape[0]):
    df0 = df0.sample(frac=1)
df0 = df0.reset_index().drop('index', axis=1)
df0.iloc[50:100]

,instruction,input,output
50,How are wines Stone?,,Smelling or tasting stones in a wine is not un...
51,Should I check out the wine list online before...,,Many good restaurants post wine lists (or “sam...
52,I don't understand wines that are Confident.,,RdV Vineyards winemaker Joshua Grainer says a ...
53,What's a good pizza for a plain or margherita ...,,Chianti Classico. A classic pizza calls for a...
54,My pizza has broccoli rabe and cherry peppers ...,,"Open a deep, cherry-red Negroamaro from Puglia..."
55,Give me a wine that pairs with stuffed shells.,,Altesino 2017 Montosoli Sangiovese (Brunello d...
56,What does it mean when a wine is Vanillin?,,Vanillin is the name for a fragrant compound t...
57,Explain Cabernet Franc.,,A lot of people think of mostly white wines wh...
58,Suggest me some cheese that pairs well with sp...,,"Hailing from New York State’s Adirondacks, Kun..."
59,Educate me on Chateauneuf-du-Pape.,,The most famous and regal of the southern Rhon...


In [6]:
# check for NaNs in dataset
for index,row in df0.iterrows():
  if (len(str(row['output'])) < 20) or (len(str(row['instruction'])) < 10):
    print(row)
df0.shape

(275, 3)

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
"""
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])
tokenized_datasets = dataset.map(tokenize_function, batched=True)
"""
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = Dataset.from_pandas(df0)
tokenizer = AutoTokenizer.from_pretrained(nn)
#dataset = load_dataset(nn, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset

# Load the base model and tokenizer
model_id = nn
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float32, device_map="auto") # Must be float32 for MacBooks!
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Load the training dataset
#dataset = load_dataset("csv", data_files="data/sarcasm.csv", split="train")
dataset = Dataset.from_pandas(df0)

# Define a function to apply the chat template
def apply_chat_template(example):
    messages = [
        {"role": "user", "content": example['instruction']},
        {"role": "assistant", "content": example['output']}
    ]
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    return {"prompt": prompt}

# Apply the chat template function to the dataset
new_dataset = dataset.map(apply_chat_template)
new_dataset = new_dataset.train_test_split(0.05) # Let's keep 5% of the data for testing

# Tokenize the data
def tokenize_function(example):
    tokens = tokenizer(example['prompt'], padding="max_length", truncation=True, max_length=128)
    # Set padding token labels to -100 to ignore them in loss calculation
    tokens['labels'] = [
        -100 if token == tokenizer.pad_token_id else token for token in tokens['input_ids']
    ]
    return tokens

# Apply tokenize_function to each row
tokenized_dataset = new_dataset.map(tokenize_function)
tokenized_dataset = tokenized_dataset.remove_columns(['question', 'answer', 'prompt'])

# Define training arguments
model.train()
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps", # To evaluate during training
    eval_steps=40,
    logging_steps=40,
    save_steps=150,
    per_device_train_batch_size=2, # Adjust based on your hardware
    per_device_eval_batch_size=2,
    num_train_epochs=2, # How many times to loop through the dataset
    fp16=True, # Must be False for MacBooks
    report_to="none", # Here we can use something like tensorboard to see the training metrics
    log_level="info",
    learning_rate=1e-5, # Would avoid larger values here
    max_grad_norm=2 # Clipping the gradients is always a good idea
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer)

# Train the model
trainer.train()

# Save the model and tokenizer
trainer.save_model("./fine-tuned-model")
tokenizer.save_pretrained("./fine-tuned-model")